In [7]:
import json
from pycocotools.coco import COCO
import tqdm

In [8]:
coco = COCO('/home/ubuntum/Downloads/annotations_trainval2017/annotations/instances_train2017.json')

loading annotations into memory...
Done (t=11.25s)
creating index...
index created!


In [9]:
# Define the list of relevant classes
relevant_classes = [1,4,13,37,44,49,63,72,77,78,79,82,84,88]

# Initialize an empty dictionary to store the bounding box values
bbox_dict = {}

# Iterate over the annotations for all images in the COCO dataset
for img in tqdm.tqdm(coco.dataset['images']):
    for ann in tqdm.tqdm(coco.dataset['annotations']):
        if  ann['image_id'] == img['id']and ann['category_id'] in relevant_classes:
            image_id = ann['image_id']
            bbox = ann['bbox']

            # Convert the bbox to YOLO format
            image_info = coco.loadImgs(image_id)[0]
            image_width = image_info['width']
            image_height = image_info['height']
            x_center = ((2*bbox[0] + bbox[2])/(2*image_width))
            y_center = ((2*bbox[1] + bbox[3])/(2*image_height))
            bbox_width = bbox[2] / image_width
            bbox_height = bbox[3] / image_height
            
            # Add the bbox values to the dictionary
            if image_id in bbox_dict:
                bbox_dict[image_id].append(f"{ann['category_id']} {x_center} {y_center} {bbox_width} {bbox_height}")
            else:
                bbox_dict[image_id] = [f"{ann['category_id']} {x_center} {y_center} {bbox_width} {bbox_height}"]
import os
# Save the bbox values to text files
for image_id in bbox_dict:
    image_info = coco.loadImgs(image_id)[0]
    image_filename = image_info['file_name']
    file_path = os.path.join('/home/ubuntum/Desktop/CocoBbox', f"{image_filename.split('.')[0]}.txt")
    with open(file_path, 'w') as f:
        for bbox_str in bbox_dict[image_id]:
            f.write(bbox_str + '\n')

100%|██████████| 10/10 [00:03<00:00,  3.07it/s]


In [21]:
# Load the category information
#categories = coco.loadCats(coco.getCatIds())

#output =os.path.join('/home/ubuntum/Desktop/CocoBbox', 'Categories.txt')
#with open(output, 'w') as f:
    #for category in categories:
        #f.write(f"Category ID: {category['id']}, Name: {category['name']}\n")

In [5]:
import cv2
import glob
import os

# Define the list of relevant classes
relevant_classes = [1,4,13,37,44,49,63,72,77,78,79,82,84,88]

# Load the text file
txt_files = sorted(glob.glob("/home/ubuntum/Desktop/CocoBbox/*.txt"))

output_folder = "/home/ubuntum/Desktop/YoloBboxImages/"

# Parse the txt_files
for txt_file in txt_files:
    with open(txt_file, 'r') as f:
        lines = f.readlines()

        # Get the image filename
        file_name = os.path.splitext(os.path.basename(txt_file))[0]
        image_file = os.path.join("/home/ubuntum/Downloads/train2017/", file_name + ".jpg")

        # Load the image
        img = cv2.imread(image_file)

        # Draw the bounding boxes
        for line in lines:

            line = line.strip()
            category_id, x_center, y_center, bbox_width, bbox_height = [float(x) for x in line.split(" ")]

            if category_id in relevant_classes:
                
                # Parse the bbox values
                bbox = [float(x) for x in line.strip().split(" ")[1:]]
                x1 = int((bbox[0] - bbox[2]/2) * img.shape[1])
                y1 = int((bbox[1] - bbox[3]/2) * img.shape[0])
                x2 = int((bbox[0] + bbox[2]/2) * img.shape[1])
                y2 = int((bbox[1] + bbox[3]/2) * img.shape[0])

                # Draw the bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Save the image
        output_file = os.path.join(output_folder, file_name + ".jpg")
        cv2.imwrite(output_file, img)

cv2.destroyAllWindows()

In [6]:
import glob

# Define the mapping of old category IDs to new ones
category_map = {
    '44': '1',
    '77': '2',
    '49': '8',
    '84': '13',
    '88': '17',
    '37': '18',
    '13': '32',
    '72': '20',
    '63': '21',
    '78': '22',
    '79': '22',
    '82': '23',
    '13': '32',
    '4' : '34',
    '1' : '36'
}

# Load the text file
txt_files = glob.glob("/home/ubuntum/Desktop/CocoBbox/*.txt")

# Parse the txt_files
for file_path in txt_files:
    with open(file_path, "rt") as fin:
        lines = fin.readlines()

    # Loop through each line and change the first word
    for i in range(len(lines)):
        words = lines[i].split()
        if len(words) > 0:
            # Replace the first word with the new category ID
            old_category_id = words[0]
            if old_category_id in category_map:
                new_category_id = category_map[old_category_id]
                words[0] = new_category_id
                lines[i] = ' '.join(words) + '\n'

    with open(file_path, "wt") as fout:
        # write new category ID
        fout.writelines(lines)
